In [1]:
import pandas as pd
import numpy as np
import json
import os

In [2]:
#train = pd.read_csv("le_dima.csv",index_col=[0])
train = pd.read_csv("../prep_train_data.csv")


In [3]:
train

,Unnamed: 0,msno,song_id,source_system_tab,source_screen_name,source_type,target,city,bd,gender,...,registration_init_time,expiration_date,registration_year,registration_month,registration_day,expiration_year,expiration_month,expiration_day,nogender_noage,membership_duration
0,0,7640,60186,1,7,7,1,1,27,3,...,2012-01-02,2017-10-05,2012,1,2,2017,10,5,True,2103
1,1,16139,179889,4,8,5,1,13,24,female,...,2011-05-25,2017-09-11,2011,5,25,2017,9,11,False,2301
2,2,16139,97284,4,8,5,1,13,24,female,...,2011-05-25,2017-09-11,2011,5,25,2017,9,11,False,2301
3,3,16139,19141,4,8,5,1,13,24,female,...,2011-05-25,2017-09-11,2011,5,25,2017,9,11,False,2301
4,4,7640,26909,1,7,7,1,1,27,3,...,2012-01-02,2017-10-05,2012,1,2,2017,10,5,True,2103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3314961,3314961,21047,12341,4,8,4,1,1,27,3,...,2017-01-05,2017-10-05,2017,1,5,2017,10,5,True,273
3314962,3314962,3956,151043,4,8,5,1,4,26,female,...,2004-12-18,2017-09-30,2004,12,18,2017,9,30,False,4669
3314963,3314963,17081,285816,7,15,9,0,1,27,3,...,2013-06-27,2017-10-06,2013,6,27,2017,10,6,True,1562
3314964,3314964,1150,82552,0,5,7,1,5,27,3,...,2015-02-13,2017-09-18,2015,2,13,2017,9,18,True,948


In [4]:

import sys
import surprise

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import (
    rmse,
    mae,
    rsquared,
    exp_var,
    map_at_k,
    ndcg_at_k,
    precision_at_k,
    recall_at_k,
    get_top_k_items,
)
from recommenders.models.surprise.surprise_utils import (
    predict,
    compute_ranking_predictions,
)
from recommenders.utils.notebook_utils import store_metadata


C:\Users\Andrey\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandera\engines\pandas_engine.py:67: UserWarning: Using typeguard < 3. Generic types like List[TYPE], Dict[TYPE, TYPE] will only validate the first element in the collection.
  warnings.warn(


In [5]:
data = train[['msno','song_id','target']]

In [6]:
data

,msno,song_id,target
0,7640,60186,1
1,16139,179889,1
2,16139,97284,1
3,16139,19141,1
4,7640,26909,1
...,...,...,...
3314961,21047,12341,1
3314962,3956,151043,1
3314963,17081,285816,0
3314964,1150,82552,1


In [7]:
import random
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

In [8]:

reader = Reader(rating_scale=(0, 1))

dataset = Dataset.load_from_df(data, reader)

In [9]:
trainset, testset = train_test_split(dataset, test_size=0.2, random_state=42)

In [41]:

from surprise import SVD, Dataset, accuracy
from surprise.model_selection import GridSearchCV, cross_validate
from surprise.reader import Reader
import pandas as pd

In [43]:
#Был произведен перебор параметров по n_factors,n_epochs,lr_all, reg_all
model = SVD(n_factors=300, n_epochs=20, lr_all=0.05, reg_all=0.03)
model.fit(trainset)

In [44]:
predictions = model.test(testset)

mae = accuracy.mae(predictions)
rmse = accuracy.rmse(predictions)

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

MAE:  0.3935
RMSE: 0.4508
MAE: 0.3935028950954084
RMSE: 0.4508491471648204


In [12]:
predictions

[Prediction(uid=23685, iid=222574, r_ui=1.0, est=0.6731664308474652, details={'was_impossible': False}),
 Prediction(uid=12061, iid=160564, r_ui=0.0, est=0.4083920611132324, details={'was_impossible': False}),
 Prediction(uid=12953, iid=240622, r_ui=0.0, est=0.46599881422035677, details={'was_impossible': False}),
 Prediction(uid=18324, iid=276690, r_ui=0.0, est=0.36836489871196054, details={'was_impossible': False}),
 Prediction(uid=2299, iid=45862, r_ui=1.0, est=0.7177738586347127, details={'was_impossible': False}),
 Prediction(uid=13235, iid=95976, r_ui=0.0, est=0.3230587467877585, details={'was_impossible': False}),
 Prediction(uid=4664, iid=124579, r_ui=0.0, est=0.8632879214702919, details={'was_impossible': False}),
 Prediction(uid=17678, iid=211995, r_ui=0.0, est=0.5032481217692979, details={'was_impossible': False}),
 Prediction(uid=65, iid=104410, r_ui=0.0, est=0.31651389168296173, details={'was_impossible': False}),
 Prediction(uid=19014, iid=12256, r_ui=0.0, est=0.265407130

In [13]:
from collections import defaultdict

from surprise import Dataset, SVD


def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [58]:
top_n = get_top_n(predictions, n=5)

In [59]:
top_n

defaultdict(list,
            {23685: [(64807, 1),
              (207096, 1),
              (15370, 1),
              (34741, 0.9968324324822325),
              (278533, 0.9703816403171002)],
             12061: [(13770, 0.7512573946529287),
              (16787, 0.7409487877008516),
              (138121, 0.7362001021206976),
              (238743, 0.735050212993972),
              (44463, 0.6544856353984485)],
             12953: [(284391, 0.9409250756197491),
              (156096, 0.9165449821910913),
              (135549, 0.8611335005486924),
              (278083, 0.8574566101920092),
              (89515, 0.8532471471111955)],
             18324: [(139636, 0.5884789723333084),
              (273560, 0.4199177038472588),
              (267260, 0.4126890012386862),
              (34328, 0.40890974070238945),
              (113016, 0.3879020417262614)],
             2299: [(62864, 1),
              (153747, 0.9969932052549472),
              (101579, 0.9847565030908056),
         

In [68]:
merged_data = train

In [17]:
X = merged_data.drop(columns=['target','registration_init_time','expiration_date'])
y = merged_data['target']

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [19]:
from catboost import CatBoostClassifier
cls = CatBoostClassifier(cat_features=['gender'],task_type='GPU')
cls.fit(X_train, y_train)

Learning rate set to 0.021929
0:	learn: 0.6907046	total: 140ms	remaining: 2m 19s
1:	learn: 0.6883723	total: 243ms	remaining: 2m 1s
2:	learn: 0.6861465	total: 351ms	remaining: 1m 56s
3:	learn: 0.6840556	total: 466ms	remaining: 1m 55s
4:	learn: 0.6820172	total: 577ms	remaining: 1m 54s
5:	learn: 0.6801079	total: 688ms	remaining: 1m 53s
6:	learn: 0.6782846	total: 800ms	remaining: 1m 53s
7:	learn: 0.6764965	total: 916ms	remaining: 1m 53s
8:	learn: 0.6747766	total: 1.03s	remaining: 1m 53s
9:	learn: 0.6731777	total: 1.14s	remaining: 1m 53s
10:	learn: 0.6716285	total: 1.25s	remaining: 1m 52s
11:	learn: 0.6701183	total: 1.35s	remaining: 1m 51s
12:	learn: 0.6687256	total: 1.43s	remaining: 1m 48s
13:	learn: 0.6673710	total: 1.53s	remaining: 1m 47s
14:	learn: 0.6660813	total: 1.63s	remaining: 1m 46s
15:	learn: 0.6648489	total: 1.73s	remaining: 1m 46s
16:	learn: 0.6636677	total: 1.83s	remaining: 1m 45s
17:	learn: 0.6624963	total: 1.93s	remaining: 1m 45s
18:	learn: 0.6614182	total: 2.03s	remaining: 

In [55]:
##Метрики
from sklearn.metrics import confusion_matrix

def metric(user_id, predicted, score = "accuracy"):
    y_true = []
    for song_id in predicted:
        if song_id in merged_data[merged_data["msno"] == user_id]['song_id'].values:
            y_true.append(merged_data[merged_data["msno"] == user_id].loc[merged_data["song_id"] == song_id]['target'].values)
        else:
            user_info = merged_data[merged_data['msno'] == user_id].iloc[0, 6:-10].T.values
            song_info = merged_data[merged_data['msno'] == song_id].iloc[0, -10:].T.values
            obj = np.array([user_id, song_id, "new", "new", "new"]+user_info+song_info)
            y_true.append(cls.predict(obj))
    if len(y_true) > 0 and  len(confusion_matrix(y_true, np.ones(len(y_true))).ravel()) == 4:
        tn, fp, fn, tp = confusion_matrix(y_true, np.ones(len(y_true))).ravel()
    else:
        return -1

    if score == "accuracy": return round((tp+tn)/(tp+tn+fp+fn), 2)
    elif score == "presicion": return round(tp/(tp+fp), 2)
    elif score == "recall": return round(tp/(tp+tn), 2)
def batch_user_metric(user_ids, predictions, score = "accuracy"):
    """
    user_ids: list/array of users
    predictions: list/array of prection for each user
    score: set metric
    """
    return [metric(user_ids[i], predictions[i], score) for i in range(len(user_ids))]/len(user_ids)

In [65]:
#Усредняем метрику по пользователям
usr_idx = list(top_n.keys())[:500]
metrics = []
for el in usr_idx:
    try:
        res = metric(el,[x[0] for x in top_n[el]])
        if res != -1:
            metrics.append(res)
    except Exception as e:
        pass

C:\Users\Andrey\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
C:\Users\Andrey\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
C:\Users\Andrey\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
C:\Users\Andrey\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classific

In [67]:
np.array(metrics).mean()

0.5435308641975308